In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# স্টক সিম্বল ও সময়সীমা নির্ধারণ করুন
stocks = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
data = yf.download(stocks, start="2020-01-01", end="2025-08-02", auto_adjust=False)
adj_close = data['Adj Close']
print(data)





[*********************100%***********************]  5 of 5 completed

Price        Adj Close                                                  \
Ticker            AAPL        AMZN       GOOGL        MSFT        TSLA   
Date                                                                     
2020-01-02   72.620857   94.900497   68.026016  153.042313   28.684000   
2020-01-03   71.914795   93.748497   67.670158  151.136673   29.534000   
2020-01-06   72.487877   95.143997   69.473846  151.527283   30.102667   
2020-01-07   72.146957   95.343002   69.339645  150.145752   31.270666   
2020-01-08   73.307518   94.598503   69.833183  152.537292   32.809334   
...                ...         ...         ...         ...         ...   
2025-07-28  214.050003  232.789993  192.580002  512.500000  325.589996   
2025-07-29  211.270004  231.009995  195.750000  512.570007  321.200012   
2025-07-30  209.050003  230.190002  196.529999  513.239990  319.040009   
2025-07-31  207.570007  234.110001  191.899994  533.500000  308.269989   
2025-08-01  202.380005  214.750000  18

In [ ]:
# দৈনিক রিটার্ন (%) বের করুন
returns = data.pct_change().dropna()
print(returns.head())  # প্রkjkhkljkjh nথম ৫ সারি দেখুনkjhkjljjkkj


Price      Adj Close                                             Close  \
Ticker          AAPL      AMZN     GOOGL      MSFT      TSLA      AAPL   
Date                                                                     
2020-01-03 -0.009723 -0.012139 -0.005231 -0.012452  0.029633 -0.009722   
2020-01-06  0.007969  0.014886  0.026654  0.002584  0.019255  0.007968   
2020-01-07 -0.004703  0.002092 -0.001932 -0.009117  0.038801 -0.004703   
2020-01-08  0.016086 -0.007809  0.007118  0.015928  0.049205  0.016086   
2020-01-09  0.021241  0.004799  0.010498  0.012493 -0.021945  0.021241   

Price                                               ...      Open            \
Ticker          AMZN     GOOGL      MSFT      TSLA  ...      AAPL      AMZN   
Date                                                ...                       
2020-01-03 -0.012139 -0.005231 -0.012452  0.029633  ...  0.003072 -0.005600   
2020-01-06  0.014886  0.026654  0.002585  0.019255  ... -0.011307 -0.002413   
2020-01-07  